In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import nltk
import json
import os
import sys
import time
import operator
#import re
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
###########
ps = PorterStemmer()

In [6]:
########### Set up Output File
outdir = '..//..//data//clean_data'

##### Contain 2 column : 1. Replaced name of entity / 2. Real name of entity
reverse_entity = open(os.path.join(outdir,os.path.basename('entity_total.csv')),'w',encoding='utf-8') 

#####
#entity_replaced = open(os.path.join(outdir,os.path.basename('entity_replaced_4.csv')),'w',encoding='utf-8')



In [7]:
########### Entity sources

#### Drug synonyms from drugbank
#drugs = json.load(open('..//data/preprocessing/drug_corpus/drug_name_synonyms.json',encoding='utf-8'))
drugs = json.load(open('..//..//data/preprocessing/drug_corpus/drugs.json',encoding='utf-8'))
#### Herb scientific name and its synonyms (not alias)
#herbs = json.load(open('..//data/preprocessing/herb_corpus/herb_science_synonyms.json',encoding='utf-8'))
herbs = json.load(open('..//..//data/preprocessing/herb_corpus/herb_syms_total.json',encoding='utf-8'))
#### standard ref without effect
ref_without = json.load(open('..//..//data/preprocessing/standard_ref/standard_ref_without_effect.json',encoding='utf-8'))

#### standard ref with effect
#ref_effect = open('..//data/preprocessing/standard_ref/standard_ref_effect.csv').read().split('\n')[:-1]

#### Adverse effect corpus
#adverse_effect = json.load(open('..//data/preprocessing/adverse_effect_corpus/adverse_effect.json'))





In [8]:
########## Create name for entity

###### trigger entity
#trigger_entity = {}
#for each in triggers :
#    trigger_entity[each] = 'trigger_entity'
#    reverse_entity.write('trigger_entity\t%s\n'%each)

###### Drug entity
drug_entity = {}
drugorder = 0
for ID in drugs :
    for drug in drugs[ID] :
        if drug.lower() in herbs or drug.lower() in ref_without :
            break
        drug_entity[drug.lower()] = 'drug_entity_%s'%drugorder
        try :
            reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drug.lower()))
        except :
            drugencode = str(drug.lower()).encode('utf8')
            reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drugencode))
    drugorder+=1

###### Herb entity 
herb_entity = {}
herborder = 0
for herb in herbs :
    #herb_entity[herb.lower()] = 'herb_entity_%s'%herborder
    reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herb.lower()))
    for synonym in herbs[herb] :
        herb_entity[synonym.lower()] = 'herb_entity_%s'%herborder
        try :
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,synonym.lower()))
        except :
            herbencode = str(synonym.lower()).encode('utf8')
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herbencode))
    herborder+=1
    
###### Adverse effect entity
#effect_entity = {}
#effectorder = 0
#for each in adverse_effect :
#    effect_entity[each.lower()] = 'EF_entity_%s'%effectorder
#    try :
#        reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,each.lower()))
#    except :
#        effectencode = str(each.lower()).encode('utf8')
#        reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effectencode))
#    effectorder+=1
    
###### standard ref without effect entity 
for herb in ref_without :
    if herb.lower() not in herb_entity :
        herb_entity[herb.lower()] = 'herb_entity_%s'%herborder
        try :
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herb.lower()))
        except :
            herbencode = str(herb.lower()).encode('utf8')
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herbencode))
        herborder+=1
    for drug in ref_without[herb] :
        if drug.lower() not in drug_entity :
            drug_entity[drug.lower()] = 'drug_entity_%s'%drugorder
            try :
                reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drug.lower()))
            except :
                drugencode = str(drug.lower()).encode('utf8')
                reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drugencode))
            drugorder+=1
            
###### standard ref with effect
''''''
#for row in ref_effect :
#    effect = row.split('\t')[2]
#    ###
#    if effect.lower() not in effect_entity :
#        effect_entity[effect.lower()] = 'EF_entity_%s'%effectorder
#        try :
#            reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effect.lower()))
#        except :
#            effectencode = str(effect.lower()).encode('utf8')
#            reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effectencode))
#        effectorder+=1
#    ###    
#    if len(row.split('\t')) == 4 :
#        effect1 = row.split('\t')[3]
#        if effect1.lower() not in effect_entity :
#            effect_entity[effect1.lower()] = 'EF_entity_%s'%effectorder
#            try :
#                reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effect1.lower()))
#            except :
#                effectencode = str(effect1.lower()).encode('utf8')
#                reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effectencode))
#            effectorder+=1
''''''
######
reverse_entity.close()

In [6]:
###### remove '\n' from text and merge text
def rmv_n(textdict) :
    begin = time.time()
    text = ''
    print('merge text')

    #text = ''.join(''.join(operator.itemgetter(*list(textdict))(textdict)).split('\n')).lower()
    
    #text = ''.join(''.join([textdict[x] for x in list(textdict)]).split('\n')).lower()
    #text =  ''.join([''.join(x.split('\n')) for sublist in textdict for x in sublist ])
    text = ''.join([textdict[x].replace('\n',' ') for x in list(textdict)])
    print(time.time() - begin)
    print('merge text done')
    return text

In [7]:
###### Entity replacement
def entity_replace(text) :
    ###
    begin = time.time()
    ###
    count = 0
    print('herb')
    for herb in herb_entity :
        if herb in text :
            text = text.replace(' %s '%herb,' %s '%herb_entity[herb])
            #re.sub(herb,herb_entity[herb],text)
            #if herb not in replaced_list :
                #entity_replaced.write('%s\t%s\n'%(herb_entity[herb],herb))
                #replaced_list.append(herb)
        if count % 10000 == 0 :
            print('%s / %s'%(count,len(herb_entity)))
        count+=1
    print('herb done')
    
    ###
    drug_entity_single = {}
    count = 0
    print('drug')
    for drug in drug_entity :
        if drug in text :
            text = text.replace(' %s '%drug,' %s '%drug_entity[drug])
            #re.sub(drug,drug_entity[drug],text)
            #if drug not in replaced_list :
                #entity_replaced.write('%s\t%s\n'%(drug_entity[drug],drug))
                #replaced_list.append(drug)
        #else:
            #drug_entity_single[drug] = drug_entity[drug]
        if count % 10000 == 0 :
            print('%s / %s'%(count,len(drug_entity)))
        count+=1
    print('drug done')
    ###
#    print('effect')
#    count = 0
#    for effect in effect_entity :
#        if effect in text :
#            text = text.replace(effect,' %s '%effect_entity[effect])
#            #re.sub(effect,effect_entity[effect],text)
#            if effect not in replaced_list :
#                entity_replaced.write('%s\t%s\n'%(effect_entity[effect],effect))
#                replaced_list.append(effect)
#        if count % 1000 == 0 :
#            print('%s / %s'%(count,len(drug_entity)))
#        count+=1
#    print('effect done')
    ###
#    for trig in trigger_entity:
#        if trig in text:
#            text = text.replace(trig,' %s '%trigger_entity[trig])
            #re.sub(trig,trigger_entity[trig],text)
#            if trig not in replaced_list :
#                entity_replaced.write('%s\t%s\n'%(trigger_entity[trig],trig))
#                replaced_list.append(effect)
#    print('trigger done')
    print(time.time() - begin)
    print('replace entities done')
    return text

In [8]:
######## word processing and word removal
#stop_words = list(set(stopwords.words('english')))
#punctuations = string.punctuation

stop_and_punctuations = list(set(stopwords.words('english'))) + [i for i in string.punctuation ]


def word_processing(text,track,stop_and_punctuations) :
    begin = time.time()
    count = 0
    outdir = '..//..//data//clean_data//noun_verb_abs'
    #outdir = '..//data//clean_data//'
    subtrack = 0
    ######
    Pos_accept = ['N','V']
    
    ###### Split text into words
    #words = nltk.pos_tag(word_tokenize(text))
    #text = ''
    sentences = [ x for x in TextBlob(text).sentences if len(x.split()) > 3 ]
    total = len(sentences)
    print('Processing %s sentences'%total)
    #######
    sentences_noun_verb = []
    ########
    sentence_noun_verb = []
    ###### Processing
    sentences_noun_verb = [ [word[0] for word in sentence.tags if word[1][0] in Pos_accept and word[0] not in stop_and_punctuations ]  for sentence in sentences ]
    
    ##### 
    print(time.time() - begin)
    print('Word processed done')

    ##### Noun verb
    #outdir = '..//data//clean_data//noun_verb'
    outfile = open(os.path.join(outdir,os.path.basename('processed_sentences_%s.json'%track)),'w')
    outfile.write(json.dumps(sentences_noun_verb))
    outfile.close()
    sentences_noun_verb = []

In [ ]:
########### Make outdir
os.system('mkdir "..//..//data//clean_data//noun_verb_abs"')
########### Text source
text_list = os.listdir('..//..//data/abstract_full_1/')[:142]
#text_list = ['abstract_00.json']
count = 0
track = 0
#replaced_list = []
textdict = {}
#for file in text_list :
#for i in range(len(text_list)) :
for file in text_list :
    textdict = json.load(open('..//../data/abstract_full_1/%s'%file))
    #####
    print('Processing loop %s'%track)
    text = rmv_n(textdict).lower()
    textdict = []
    #####
    print('step 1')
    text = entity_replace(text)
    #####
    print('step 2')
    sentences_processed = word_processing(text,track,stop_and_punctuations)
    text = ''
    #####
    ##### 
    track+=1

#########


#####
#entity_replaced.close()

Processing loop 0
merge text
0.021001100540161133
merge text done
step 1
herb
0 / 61577
10000 / 61577
20000 / 61577
30000 / 61577
40000 / 61577
50000 / 61577
60000 / 61577
herb done
drug
0 / 26739
10000 / 26739
20000 / 26739
drug done
703.569712638855
replace entities done
step 2
Processing 55166 sentences
1524.6215860843658
Word processed done
Processing loop 1
merge text
0.028001785278320312
merge text done
step 1
herb
0 / 61577
10000 / 61577
20000 / 61577
30000 / 61577
40000 / 61577
50000 / 61577
60000 / 61577
herb done
drug
0 / 26739
10000 / 26739
20000 / 26739
drug done
568.9988331794739
replace entities done
step 2
Processing 49785 sentences
1594.3957698345184
Word processed done
Processing loop 2
merge text
0.03800487518310547
merge text done
step 1
herb
0 / 61577
10000 / 61577
20000 / 61577
30000 / 61577
40000 / 61577
50000 / 61577
60000 / 61577
herb done
drug
0 / 26739
10000 / 26739
20000 / 26739
drug done
767.9256677627563
replace entities done
step 2
Processing 50376 sentenc

In [11]:
textdict = []